# Домашняя работа 2

## Регрессия

С 1ого октября отменяется НДС на бриллианты, теперь они становятся новым инвестиционным инструментом. Давайте сделаем модель ценообразования для них.

Скачайте датасет diamonds.csv

В нем представлены характеристики бриллиантов и их цены с сайта jamesallen (B2C площадка) с 2022-07-01

**Описание полей**


* fluor - флуорисценуия (свойство камня светиться в лучах ультр)
* symmetry - показатель симметричности
* platform - название платформы, где был размещен камень
* quality_group - составной показатель из cut polish symmetry
* size_group - каратно весовая группа
* big_size_group - каратно-весовая группа
* shape - форма
* color - цвет
* clarity - чистота
* cut - качество огранки (может быть только у круглых камней)
* polish - полировка
* id - номер камня
* date - дата
* price - цена
* carat - кол-во карат
* tablepercent - размер площадки по отношению ширине
* price_per_carat - цена за карат
* z - длина (diameter)
* x - ширина
* depth_perc - отношение высоты к ширине
* y - высота



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/'

Mounted at /content/drive


In [ ]:
df = pd.read_csv(path + 'diamonds.csv', index_col=0)

In [ ]:
df

,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
135269,NONE,EX,jamesallen,PS,F,SI1,NaN,EX,13870838,202206,12850.0,1.55,8290.32,10.29,6.41,62.0,3.97420
48477,MED,EX,jamesallen,RD,H,VVS2,EX,EX,11725253,202207,7510.0,1.02,7362.75,6.41,6.45,62.5,4.03125
236786,NONE,EX,jamesallen,EM,H,IF,NaN,EX,14444347,202205,21220.0,2.01,10557.21,8.60,6.37,65.0,4.14050
235781,NONE,EX,jamesallen,RD,E,VS2,EX,EX,14438434,202207,8660.0,1.00,8660.00,6.39,6.44,61.4,3.95416
277744,NONE,VG,jamesallen,RD,F,VS1,VG,EX,14615276,202206,8480.0,1.00,8480.00,6.28,6.36,62.3,3.96228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20595,NONE,EX,jamesallen,RD,D,VS2,EX,EX,7535786,202207,8700.0,0.97,8969.07,6.36,6.32,62.2,3.93104
282754,NONE,EX,jamesallen,RD,D,VS2,EX,EX,14631732,202206,6650.0,0.90,7388.89,6.13,6.18,61.7,3.81306
141550,NONE,EX,jamesallen,RD,J,SI2,EX,EX,13932645,202205,11570.0,1.52,7611.84,7.36,7.39,61.0,4.50790
71237,STG,EX,jamesallen,RD,G,VS1,EX,EX,12699310,202207,8220.0,1.09,7541.28,6.55,6.60,62.0,4.09200


Необходимо предсказать `price_per_carat`

### Задание 1: Очистка

Не все камни успевают продаться за один месяц, поэтому в таблице есть повторы. Объедините данные по одному камню: подумайте, как лучше это сделать, какую цену брать.

**Попробуйте найти аномалии:** вдруг цена на некоторые камни сильно меняется (то есть продавец сам не знает, по какой цене их продавать). Также убедитесь, что остальные параметры камня не меняются.

In [ ]:
df.id.value_counts().head(3)

13649794    6
13639276    6
10341774    5
Name: id, dtype: int64

In [ ]:
id_stat = df.groupby(by='id')['carat'].count().to_frame() #Посчитайте статистику по кол-ву камней
id_stat

,carat
id,
0,4
19423,3
19449,2
19476,1
19501,3
...,...
15418480,1
15418482,1
15418486,1


In [ ]:
id_stat = df.groupby(by='id')['carat'].count().to_frame() #Посчитайте статистику по кол-ву камней

In [ ]:
# разделите выборку на две части
one = id_stat[id_stat['carat'] == 1].index.to_list()
more_than_one = id_stat[id_stat['carat'] > 1].index.to_list()

In [ ]:
df_one = df.query('id == @one')  # те камни, которые встречались один раз
df_three = df.query('id == @more_than_one') # те камни, которые встречались 2 или 3 раза

Так как мы ищем  аномальные значения, разделил выборку на камни, встречающиеся один раз и на более чем один.


In [ ]:
df_three.groupby('id')[['price', 'carat','price_per_carat','depth_perc', 'z', 'x', 'y']].agg(np.std)

,price,carat,price_per_carat,depth_perc,z,x,y
id,,,,,,,
0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
19423,334.863156,0.0,318.916742,0.404145,0.0,0.0,0.026835
19449,494.974747,0.0,423.054916,0.000000,0.0,0.0,0.000000
19501,831.384388,0.0,415.692194,0.346410,0.0,0.0,0.027436
19504,0.000000,0.0,0.000000,0.212132,0.0,0.0,0.014213
...,...,...,...,...,...,...,...
15409696,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
15410303,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
15410668,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000


In [ ]:
stat = df_three.groupby('id')[['price', 'carat','price_per_carat','depth_perc', 'z', 'x', 'y']].agg(np.std) # Посчитайте стандартное отклонение по нескольким полям,
#  подумайте где оно должно равняться 0, а где меняться в каких-то разумных пределах

In [ ]:
stat_list = stat.query('carat == 0 & depth_perc == 0 & z == 0').index.to_list()

Стандартное отклонение должно быть постоянным там, где значения не должны изменяться со временем. Можно предположить, что размеры алмазов не должны меняться со временем, поэтому смотрим аномальные камни по следующим признакам: carat, depth_perc, z. Из описания признаков - признак depth_perc имеет прямую корреляцию с x и y, поэтому их можно отбросить.

In [ ]:
df_three.query('id == 0')

,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
202754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Удалите аномальные наблюдения
df_three = df_three.query("id == @stat_list & id != 0").sort_values(by=['id','date'])

In [ ]:
df_three = df_three.groupby('id').agg('max').reset_index() #тепреь в качестве цены возьмем последнее значение по времени

df = pd.concat([df_one, df_three]).reset_index(drop=True) # соединяем результаты

Цены на бриллианты достаточно сильно меняются, попробуйте вычислить коэффициент инфляции и привести цены к последнему месяцу. Стоит учесть, что цены на все бриллианты не изменяются синхронно, то есть изменение в определенных группах может быть разным.

*Определите эти группы и рассчитайте коэффициенты инфляции для каждой из них. Подправьте цены на эти коэффициенты и создайте новую переменную.*

In [ ]:
df_three

,id,fluor,symmetry,platform,shape,color,clarity,cut,polish,date,price,carat,price_per_carat,z,x,depth_perc,y
0,19449,NONE,EX,jamesallen,RD,D,VVS2,EX,EX,202207,17550.0,1.17,15000.00,6.79,6.75,62.0,4.18500
1,19507,NONE,VG,jamesallen,EM SQ,E,VS1,NaN,VG,202207,27560.0,2.03,13576.35,7.20,6.86,70.1,4.80886
2,19518,NONE,VG,jamesallen,RD,E,VS1,EX,VG,202207,43650.0,2.11,20687.20,8.31,8.29,60.1,4.98229
3,19635,FNT,VG,jamesallen,RAD,E,VS1,NaN,EX,202207,5690.0,1.02,5578.43,5.93,5.14,70.0,3.59800
4,19656,NONE,G,jamesallen,RAD,D,VS2,NaN,EX,202207,5600.0,1.07,5233.64,5.76,5.51,74.2,4.08842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20054,15409696,NONE,EX,jamesallen,RD,I,SI1,EX,EX,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592
20055,15410303,MED,EX,jamesallen,OV,H,VVS2,NaN,EX,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000
20056,15410668,NONE,VG,jamesallen,EM,D,VS1,NaN,EX,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294
20057,15410717,MED,EX,jamesallen,EM,H,SI2,NaN,EX,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975


In [ ]:
categories = ['color', 'clarity', 'carat'] #определите по каким группам отпределять инфляцию (к примеру можно добавить каратную группу)

In [ ]:
df_index = df.groupby(['date'] + categories)[['price_per_carat']]\
             .mean()\
             .reset_index() # индекс цен
             
date_max = df_index.date.max()

In [ ]:
df_index = df_index.merge(df_index.query('date == @date_max')[categories + ['price_per_carat']]\
                          .rename(columns={'price_per_carat': 'price_per_carat_max'}), 
                          on=categories, how='outer') # сопоставляем группы с максимальной датой

In [ ]:
df_index['inflation'] = df_index['price_per_carat_max'] / df_index.price_per_carat # вычисляем инфляцию

In [ ]:
df_with_inf = df.merge(df_index[['date'] + categories + ['inflation']], 
                        on=['date'] + categories, how='left') # соединяем все в одной таблице

In [ ]:
df_with_inf['inflation'].fillna(1, inplace = True)

### Задание 2: Модель

Определите функцию потерь (MSE или MAE) и аргументируйте выбор.
Попробуйте сделать baseline.
Используйте LableEncoder для категориальных фичей и **постройте линейную модель**.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error as mae
from sklearn.linear_model import LinearRegression

lbl_e_df = df_with_inf.copy()

In [ ]:
cat_col = lbl_e_df.select_dtypes(include=['object']).columns

In [ ]:
for col in cat_col:
  lbl_e_df[col] = lbl_e_df[col].astype('category').cat.codes

Выбираем MAE, так как данная метрика более устойчива к выбросам. Так же, минимизируя данную метрику мы сможем предсказать более точную цену на большинстве объектов.

In [ ]:
X = lbl_e_df.drop(columns=["price_per_carat"], axis = 1)
y = lbl_e_df["price_per_carat"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
linear = LinearRegression()

In [ ]:
linear.fit(X_train, y_train)
y_pred = linear.predict(X_valid)

In [ ]:
label_mae = mae(y_valid, y_pred)
label_mae

1244.947743375925

Теперь попробуйте OHE или TargetEncoder (сравните их).

Нормализуйте данные.

Поработайте с пропусками (обратите внимание на то, что у fluor возможен пропуск значения, а возможно отсутствие флуоресценции)

Покажите, насколько получилось улучшить результат.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler

df_with_inf['cut'].fillna("NONE", inplace = True)

oneh_df = df_with_inf.copy()

In [ ]:
df_with_inf[cat_col].nunique()

fluor        4
symmetry     4
platform     1
shape       10
color       10
clarity      9
cut          4
polish       3
dtype: int64

In [ ]:
oneh_df = pd.get_dummies(oneh_df, drop_first=True)

In [ ]:
oneh_X = oneh_df.drop(columns=["price_per_carat"], axis = 1)
oneh_y = oneh_df["price_per_carat"]

o_X_train, o_X_valid, o_y_train, o_y_valid = train_test_split(oneh_X, oneh_y, test_size=0.2, random_state=13)

In [ ]:
scaler = StandardScaler() # Помните, что на тесте делаем только transform

In [ ]:
scaler.fit(o_X_train)
o_X_train_scaled = scaler.transform(o_X_train)
o_X_test_scaled = scaler.transform(o_X_valid)

In [ ]:
import time

linear_start = time.time()

linear.fit(o_X_train_scaled, y_train)
y_pred = linear.predict(o_X_test_scaled)

linear_stop = time.time()
print('Время выполнения:', round(linear_stop -  linear_start,1))

Время выполнения: 0.4


In [ ]:
oneh_mae = mae(y_valid, y_pred)
oneh_mae

849.5046386516367

МАЕ метрика улучшилась на 30%

In [ ]:
target_df = df_with_inf.copy()

X = target_df.drop(columns=["price_per_carat"], axis = 1)
y = target_df["price_per_carat"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
target = TargetEncoder()

In [ ]:
target.fit(X_train, y_train)


TargetEncoder(cols=['fluor', 'symmetry', 'platform', 'shape', 'color',
                    'clarity', 'cut', 'polish'])

In [ ]:
X_train_scaled = target.transform(X_train)
X_test_scaled = target.transform(X_valid)

scaler.fit(X_train_scaled)
X_train_scaled = scaler.transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

In [ ]:
linear.fit(X_train_scaled, y_train)
y_pred = linear.predict(X_test_scaled)

In [ ]:
target_mae = mae(y_valid, y_pred)
target_mae

1037.3567427426806

In [ ]:
print('MAE label encoder: {} \nMAE OneHot encoder: {} | Percentage: {} \nMAE Target encoder: {} | Percentage {}'.format(
    label_mae, 
    oneh_mae,
     round((oneh_mae - label_mae)/label_mae * -100, 1), 
     target_mae, 
     round((target_mae - label_mae)/label_mae * -100, 1)
     )
)

MAE label encoder: 1244.947743375925 
MAE OneHot encoder: 849.5046386516367 | Percentage: 31.8 
MAE Target encoder: 1037.3567427426806 | Percentage 16.7


Target encoder отработал хуже, чем OneHot

Сравните работу выбранной вами линейной модели с KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=5)

In [ ]:
knn_start = time.time()

knn.fit(o_X_train_scaled, y_train)
y_pred = knn.predict(o_X_test_scaled)

knn_stop = time.time()
print('Время выполнения:', round(knn_stop -  knn_start,1))

Время выполнения: 79.7


In [ ]:
knn_mae = mae(y_valid, y_pred)
knn_mae

828.5600997493925

In [ ]:
print("MAE linear: {} | Время выполнения: {} \nMAE knn: {} | Время выполнения: {}".format(
    round(oneh_mae, 2),
    str(round(linear_stop -  linear_start, 1)) + ' sec',
    round(knn_mae, 2),
    str(round(knn_stop -  knn_start, 1)) + ' sec',
     )
)

MAE linear: 849.5 | Время выполнения: 0.4 sec 
MAE knn: 828.56 | Время выполнения: 79.7 sec


KNN оказался немного точнее, однако врмя на расчет оказалось значительно выше

## Классификация

Загрузите датасет bodyPerformance

Описание:

https://www.kaggle.com/datasets/kukuroo3/body-performance-data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = 'drive/MyDrive/'

In [ ]:
df = pd.read_csv(path + '/bodyPerformance.csv') #укажите свой путь

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
df

,age,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,gender_M,class_B,class_C,class_D
0,27.0,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,1,0,1,0
1,25.0,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,1,0,0,0
2,31.0,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,1,0,1,0
3,32.0,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,1,1,0,0
4,28.0,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,1,0,1,0
13389,21.0,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,1,0,0,1
13390,39.0,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,1,0,0,0
13391,64.0,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,0,0,0,1


### Задание 1: Определение гендера

Постройте модель, которая будет определять гендер.

Для этого определите две метрики качества, на которые вы будете смотреть. Аргументируйте свой выбор.

После чего преобразуйте категориальные переменные и постройте модель бинарной классификации на основе линейного классификатора и сравните с KNN.

P.S.: не забудте про нормализацию.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  f1_score, roc_curve, auc

X = df.drop(columns=["gender_M"], axis = 1)
y = df["gender_M"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

scaler = StandardScaler()

In [ ]:
scaler.fit(X_train)

scaler.transform(X_train)
scaler.transform(X_valid)

array([[-0.50344497,  1.12141406,  0.35110283, ...,  1.74136938,
        -0.58015243, -0.57598512],
       [-1.09144916,  1.1095389 ,  1.15503999, ..., -0.5742607 ,
         1.72368492, -0.57598512],
       [-0.35644392,  0.50390602,  0.60233319, ..., -0.5742607 ,
        -0.58015243, -0.57598512],
       ...,
       [-1.16494968,  1.19266499,  0.45159498, ..., -0.5742607 ,
        -0.58015243,  1.73615597],
       [-0.42994445,  1.41829292,  1.89198238, ...,  1.74136938,
        -0.58015243, -0.57598512],
       [ 1.18706707, -2.42925718, -1.62524268, ..., -0.5742607 ,
        -0.58015243, -0.57598512]])

In [ ]:
logi = LogisticRegression()

In [ ]:
logi.fit(X_train, y_train)

y_pred_train = logi.predict(X_train)
y_pred_test = logi.predict(X_valid)
y_pred_test_prob = logi.predict_proba(X_valid)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Так как у нас довольно абстрактная задача классификации, в качестве метрик возьмем среднегармоническое точности и полноты (f1-score).

In [ ]:
print(f"F1 Train: {f1_score(y_train, y_pred_train)}")
print(f"F1 Test: {f1_score(y_valid, y_pred_test)}")

F1 Train: 0.987431613189413
F1 Test: 0.989209682123068


In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors = 10)

In [ ]:
knn_clf.fit(X_train, y_train)

knn_y_pred_train = knn_clf.predict(X_train)
knn_y_pred_test = knn_clf.predict(X_valid)

In [ ]:
print(f"F1 Train: {f1_score(y_train, knn_y_pred_train)}")
print(f"F1 Test: {f1_score(y_valid, knn_y_pred_test)}")

F1 Train: 0.9854813491177128
F1 Test: 0.987980064497215


### Задание 2: Определение класса

Теперь опробуем построить модель, которая будет предсказывать class физической формы.

Для этого определите метрики качества для задачи мультиклассификации (аргументируйте выбор).

Постройте модель мультиклассовой классификации на основе линейного классификатора и сравните с KNN.

In [ ]:
df = pd.read_csv(path + '/bodyPerformance.csv') #укажите свой путь

In [ ]:
df.replace({"gender": {"M": 1, "F": 0}}, inplace = True)

In [ ]:
X = df.drop(columns=["class"]) # укажите новый таргет
y = df["class"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
scaler.fit(X_train)

scaler.transform(X_train)
scaler.transform(X_valid)

array([[-0.50344497, -1.30649251,  1.12141406, ...,  1.45906131,
        -0.68075408,  0.19914104],
       [-1.09144916,  0.76540814,  1.1095389 , ..., -0.4407566 ,
         0.57726913,  1.4767606 ],
       [-0.35644392,  0.76540814,  0.50390602, ...,  0.10887625,
         1.34606108,  1.45170923],
       ...,
       [-1.16494968,  0.76540814,  1.19266499, ...,  0.65850911,
        -0.19152283, -0.30188624],
       [-0.42994445,  0.76540814,  1.41829292, ...,  0.22836165,
         0.78693966,  1.50181196],
       [ 1.18706707, -1.30649251, -2.42925718, ...,  0.99306823,
        -0.75064426, -1.17868397]])

In [ ]:
logi.fit(X_train, y_train)

y_pred_train = logi.predict(X_train)
y_pred_test = logi.predict(X_valid)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
print(f"F1 Train macro: {f1_score(y_train, y_pred_train, average='macro')}")
print(f"F1 Test macro: {f1_score(y_valid, y_pred_test, average='macro')}")
print()
print(f"F1 Train micro: {f1_score(y_train, y_pred_train, average='micro')}")
print(f"F1 Test micro: {f1_score(y_valid, y_pred_test, average='micro')}")

F1 Train macro: 0.6000867998626103
F1 Test macro: 0.5949695503522044

F1 Train micro: 0.6032294194511854
F1 Test micro: 0.5968645016797313


In [ ]:
knn_clf.fit(X_train, y_train)

knn_y_pred_train = knn_clf.predict(X_train)
knn_y_pred_test = knn_clf.predict(X_valid)

In [ ]:
print(f"F1 Train macro: {f1_score(y_train, knn_y_pred_train, average='macro')}")
print(f"F1 Test macro: {f1_score(y_valid, knn_y_pred_test, average='macro')}")
print()
print(f"F1 Train micro: {f1_score(y_train, knn_y_pred_train, average='micro')}")
print(f"F1 Test micro: {f1_score(y_valid, knn_y_pred_test, average='micro')}")

F1 Train macro: 0.6654926168734832
F1 Test macro: 0.5780224709886159

F1 Train micro: 0.6652977412731006
F1 Test micro: 0.5755879059350504


Для предсказания в многоклассовой классификации модели справились хуже. Различия в подходах подсчета оказались незнечительными.